# Slash Product Notebook
This Jupyter notebook was created by Ryan Malthaner with algorithms from Garrett Tresch as a way of better visualizing, drawing, and working with slash products of st-graphs.  Current functionality allows for the slashing of arbitrary st-graphs together, as well as computing slash powers of a graph with itself.  All that is needed for this functionality is a numpy matrix of the adjacency matrix of the graph (be sure to put 0's on the diagonal), then the functions here will do the rest.  

The function "create_graph" creates the graph from the given adjancency matrix and returns a networkx Graph object which has a multitude of algorithms and functionality built in that you can read about <a href="https://networkx.org/documentation/stable/reference/introduction.html">here</a>.

The function "draw_graph" draws the graph in a way where the nodes can be manipulated in the window where the graph is drawn.  These node positions can then be reused as shown in the code below, so that one does not have to constantly readjust the nodes time after time.  By default the positions of the nodes are printed out to the notebook so they can be reused between launches of the notebook.

The function "comp_slash_power_verts" computes the number of vertices for a slash power of a given st-graph.  This formula is given by $|V_G| + |E_G|\cdot(|V_H| - 2)$.

The function "comp_slash_verts" computes the number of vertices for a slash product of two graphs.  This formula is given by $2 + (V - 2)\cdot\frac{E^{n} - 1}{E - 1}$.

The function "slash_H_into_G" takes an st-graph $G$ and replaces every edge of $G$ with a copy of $H$ i.e. it returns $G ⊘ H$.

The function "slash_power" takes an st-graph $G$ and performs the slash product of $G$ with itself $n$ times.  This is given by $G^{⊘(n)} = G^{⊘(n - 1)}⊘G$.  If one needs to compute multiple slash powers and retain them, it is much faster and better to use "slash_H_into_G" multiple times, as it will only compute exactly what is needed, whereas "slash_power" computes all the slash powers up until the iteration requested each time.

Please enjoy this notebook, and we hope it is productive for your research.  Any requests for features, updates, or bugs can be sent to ryan_malthaner at tamu dot edu.  Thanks!

In [1]:
import numpy as np
import networkx as nx
from netgraph import InteractiveGraph
import matplotlib.pyplot as plt

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
def create_graph(adj_matr):
    return nx.from_numpy_array(adj_matr)

def draw_graph(G, positions = None, with_labels=True, print_node_positions=True):
    #Taken from https://stackoverflow.com/questions/56084955/how-can-i-manually-place-networkx-nodes-using-the-mouse
    # decide on a layout
    if positions is None:
        positions = nx.spring_layout(G)

    # Create an interactive plot.
    # NOTE: you must retain a reference to the object instance!
    # Otherwise the whole thing will be garbage collected after the initial draw
    # and you won't be able to move the plot elements around.
    plot_instance = InteractiveGraph(G, node_layout=positions, node_labels=with_labels, scale = (2,2))
    
    #prints the node positions so they can be saved and accessed later
    if print_node_positions:
        print(copy_pastable_node_positions(plot_instance.node_positions))
        
    return plot_instance

def copy_pastable_node_positions(node_positions):
    for node in node_positions:
        node_positions[node] = list(node_positions[node])
        
    return node_positions

In [4]:
#Computes expected number of vertices for a slash power of a given st-graph
def comp_slash_power_verts(initial_vertices, initial_edges, iteration_number):
    return int(2 + (initial_vertices - 2) * (initial_edges**iteration_number - 1)/(initial_edges - 1))

#Computes expected number of vertices for a slash product of two graphs
def comp_slash_verts(G_verts, G_edges, H_verts, H_edges):
    return G_verts + G_edges*(H_verts - 2)

#Computes the slash product G \ H, replacing every edge of G with a copy of H 
#Returns a networkx Graph object
def slash_H_into_G(G, H):
    G_edges = list(G.edges)
    G_verts = G.number_of_nodes()

    H_adj_matr = nx.to_numpy_array(H)
    H_verts = H.number_of_nodes() - 2 #Number of vertices added to the new graph at every edge of G
    
    num_verts = comp_slash_verts(G_verts, len(G_edges), H_verts + 2, len(list(H.edges)))
    
    A = np.zeros([num_verts, num_verts]) #initialized new adjacency matrix
    start_index = G.number_of_nodes() #starting label number for new nodes, note that we preserve previous nodes
    
    #Loop through edges in G
    for edge in G_edges:
        #Make sure edges are ordered from smallest to largest, though it doesn't really matter
        if edge[0] > edge[1]:
            edge = (edge[1], edge[0])
            
        #Get the indices of the submatrix block, starting with the first vertex of the edge in G that you're replacing
        #with H
        block_indices = [edge[0]]
        
        #Grab the next labels which are available
        for i in range(start_index, start_index + H_verts):
            block_indices.append(i)
            start_index += 1
            
        #Grab the last vertex of the edge
        block_indices.append(edge[1])
        
        #Replace the block indexed by these matrices by the adjacency matrix of H
        A[np.ix_(block_indices, block_indices)] = H_adj_matr
    
    slash_product = nx.from_numpy_array(A)
    
    return slash_product

#Computes the slash product of a graph G with itself n times
#Returns a networkx Graph object
#Note: If you plan on computing a bunch of slash powers, I recommend manually using slash_H_into_G over and over
#as this function will do a LOT of repeated work if you do slash_power(G, 1), slash_power(G, 2), etc.
def slash_power(G, n):
    G_i = G
    
    for i in range(1, n):
        G_i = slash_H_into_G(G_i, G)
        
    return G_i

In [5]:
#adjacency matrix of the diamond graph D_1
D1_matrix = np.array([[0, 1, 1, 0],
              [1, 0, 0, 1],
              [1, 0, 0, 1],
              [0, 1, 1, 0]])

#creates graph object from the adjacency matrix
D1 = create_graph(D1_matrix)

#Computes the second diamond graph using a slash product of D_1 \ D_1
D2 = slash_power(D1, 2)

#Call the draw function once, grab the plot instance, then use plot_instance.node_positions to graph the node_positions and
#use that information in future calls to draw_graph, so you don't have to keep moving the graph every time
plot_instance = draw_graph(D2)

{0: [-0.2978174158747801, -0.7989227578936241], 1: [0.7918767773786299, -0.2991130812348783], 2: [-0.7947014329714994, 0.2976558971283099], 3: [0.3026340434955666, 0.7989622307146271], 4: [0.44770367821526474, -0.9930546729174373], 5: [0.3145607283679342, -0.6974618617982149], 6: [-0.6967114288501409, -0.3156913750381037], 7: [-0.9936835378890116, -0.45615584730185477], 8: [1.0, 0.44326154256384376], 9: [0.6935357943197923, 0.3279926086905566], 10: [-0.30986177823029654, 0.7001929358912675], 11: [-0.45753542796145913, 0.9923343811955079]}


In [6]:
#Call like this every time after you set your nodes to keep drawing them in the same place
plot_instance = draw_graph(D2, positions=plot_instance.node_positions, with_labels=True)

{0: [-0.2978174158747801, -0.7989227578936241], 1: [0.7918767773786299, -0.2991130812348783], 2: [-0.7947014329714994, 0.2976558971283099], 3: [0.3026340434955666, 0.7989622307146271], 4: [0.44770367821526474, -0.9930546729174373], 5: [0.3145607283679342, -0.6974618617982149], 6: [-0.6967114288501409, -0.3156913750381037], 7: [-0.9936835378890116, -0.45615584730185477], 8: [1.0, 0.44326154256384376], 9: [0.6935357943197923, 0.3279926086905566], 10: [-0.30986177823029654, 0.7001929358912675], 11: [-0.45753542796145913, 0.9923343811955079]}
